In [2]:
import json

with open('proj5_params.json', 'r') as f:
    params = json.load(f)

params

{'original_frequency': 'd',
 'target_frequency': 'w',
 'downsample_periods': 3,
 'downsample_units': 'd',
 'upsample_periods': 2,
 'upsample_units': 'h',
 'interpolation': 'polynomial',
 'interpolation_order': 3,
 'sensors_periods': 10,
 'sensors_units': 's'}

In [26]:
import pandas as pd

df = pd.read_csv('proj5_timeseries.csv')
df['date'] = pd.to_datetime(df['Date'])
df.index = df['date']
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace('[^a-z]', '_')
df = df.drop('date', axis=1)
df

/var/folders/p3/1mlnmr7563b2lsdz5xhg4k7w0000gn/T/ipykernel_43408/3816036359.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[^a-z]', '_')


,consumption,wind,solar,wind_solar
date,,,,
2006-01-01,1069.18400,NaN,NaN,NaN
2006-01-02,1380.52100,NaN,NaN,NaN
2006-01-03,1442.53300,NaN,NaN,NaN
2006-01-04,1457.21700,NaN,NaN,NaN
2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...
2017-12-27,1263.94091,394.507,16.530,411.037
2017-12-28,1299.86398,506.424,14.162,520.586
2017-12-29,1295.08753,584.277,29.854,614.131


In [4]:
df = df.asfreq(params['original_frequency'])

In [5]:
import pickle

with open('proj5_ex01.pkl', 'wb') as f:
    pickle.dump(df, f)

In [6]:
df_target_fq = df.asfreq(params['target_frequency'])
df_target_fq

,consumption,wind,solar,wind_solar
date,,,,
2006-01-01,1069.18400,NaN,NaN,NaN
2006-01-08,1207.98500,NaN,NaN,NaN
2006-01-15,1265.47500,NaN,NaN,NaN
2006-01-22,1248.11100,NaN,NaN,NaN
2006-01-29,1252.21500,NaN,NaN,NaN
...,...,...,...,...
2017-12-03,1330.26226,387.490,12.411,399.901
2017-12-10,1318.10964,622.944,9.922,632.866
2017-12-17,1297.21916,250.726,12.620,263.346


In [7]:
with open('proj5_ex02.pkl', 'wb') as f2:
    pickle.dump(df_target_fq, f2)

In [18]:
interval = str(params['downsample_periods']) + params['downsample_units']
df_downsampled = df.resample(interval).sum(min_count=params['downsample_periods'])
df_downsampled

,consumption,wind,solar,wind_solar
date,,,,
2006-01-01,3892.23800,NaN,NaN,NaN
2006-01-04,4337.77500,NaN,NaN,NaN
2006-01-07,4037.59500,NaN,NaN,NaN
2006-01-10,4707.36700,NaN,NaN,NaN
2006-01-13,4170.42200,NaN,NaN,NaN
...,...,...,...,...
2017-12-17,4462.39225,484.667,37.310,521.977
2017-12-20,4503.29557,578.692,26.177,604.869
2017-12-23,3525.21153,2148.306,34.164,2182.470


In [19]:
with open('proj5_ex03.pkl', 'wb') as f3:
    pickle.dump(df_downsampled, f3)

In [20]:
interval = str(params['upsample_periods']) + params['upsample_units']
df_upsampled = df.resample(interval).interpolate(method=params['interpolation'], order=params['interpolation_order'])
df_upsampled

,consumption,wind,solar,wind_solar
date,,,,
2006-01-01 00:00:00,1069.184000,NaN,NaN,NaN
2006-01-01 02:00:00,1110.188591,NaN,NaN,NaN
2006-01-01 04:00:00,1148.017321,NaN,NaN,NaN
2006-01-01 06:00:00,1182.801502,NaN,NaN,NaN
2006-01-01 08:00:00,1214.672446,NaN,NaN,NaN
...,...,...,...,...
2017-12-30 16:00:00,1141.409085,758.085640,5.086892,763.172532
2017-12-30 18:00:00,1132.454004,753.230038,7.516226,760.746264
2017-12-30 20:00:00,1123.721991,745.595267,10.765412,756.360679


In [27]:
original_timedelta = pd.Timedelta(value=1, unit=params['original_frequency'])
upsampled_timedelta = pd.Timedelta(value=params['upsample_periods'], unit=params['upsample_units'])

df_upsampled *= upsampled_timedelta/original_timedelta
df_upsampled

,consumption,wind,solar,wind_solar
date,,,,
2006-01-01 00:00:00,2.474963,NaN,NaN,NaN
2006-01-01 02:00:00,2.569881,NaN,NaN,NaN
2006-01-01 04:00:00,2.657448,NaN,NaN,NaN
2006-01-01 06:00:00,2.737966,NaN,NaN,NaN
2006-01-01 08:00:00,2.811742,NaN,NaN,NaN
...,...,...,...,...
2017-12-30 16:00:00,2.642151,1.754828,0.011775,1.766603
2017-12-30 18:00:00,2.621421,1.743588,0.017399,1.760987
2017-12-30 20:00:00,2.601208,1.725915,0.024920,1.750835


In [28]:
with open('proj5_ex04.pkl', 'wb') as f4:
    pickle.dump(df_upsampled, f4)

In [52]:
sensors_df = pickle.load(open('proj5_sensors.pkl', 'rb'))
sensors_df.head()

,device_id,value
timestamp,,
2022-11-25 00:00:25,25,6.69375
2022-11-25 00:01:25,25,6.72500
2022-11-25 00:02:25,25,6.75625
2022-11-25 00:03:25,25,6.78750
2022-11-25 00:04:25,25,6.81875


In [53]:
sensors_df = sensors_df.pivot(columns='device_id', values='value')
sensors_df

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-11-25 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:04,NaN,NaN,NaN,5.6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.133333,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-27 23:59:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.584211,NaN,NaN,NaN,NaN
2022-11-27 23:59:51,NaN,NaN,NaN,NaN,NaN,6.094737,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
interval = str(params['sensors_periods']) + params['sensors_units']
new_index = pd.date_range(sensors_df.index.round(interval).min(), sensors_df.index.round(interval).max(), freq=interval)
# sensors_df = sensors_df.resample(interval).mean()
# sensors_df

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.2375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-27 23:59:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-27 23:59:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-27 23:59:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.584211,NaN,NaN,NaN,NaN


In [67]:
sensors_unified_index = sensors_df.reindex(new_index.union(sensors_df.index)).interpolate()

In [69]:
sensors_reindexed = sensors_unified_index.reindex(new_index)

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:10,NaN,NaN,NaN,5.598599,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:20,NaN,NaN,NaN,5.596731,6.737542,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,3.237500,NaN,7.702614,NaN,NaN,NaN,6.798051,NaN
2022-11-25 00:00:30,6.698380,NaN,NaN,5.593462,6.745791,NaN,5.438030,NaN,NaN,5.367901,...,7.070370,6.500000,3.240741,NaN,7.708715,NaN,NaN,7.101559,6.791228,7.727778
2022-11-25 00:00:40,6.701852,5.085926,NaN,5.592061,6.749327,NaN,5.437879,7.545267,NaN,5.358642,...,7.072222,6.500000,3.242130,NaN,7.711329,NaN,NaN,7.103899,6.788304,7.729293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:20,4.304678,4.484096,5.268627,4.721053,6.380952,6.097661,5.159226,5.188889,4.248889,5.887800,...,5.600000,4.515789,2.300000,5.149383,5.300000,5.576023,4.697856,6.203472,4.005882,6.001235
2022-11-27 23:59:30,4.305263,4.482571,5.282353,4.721053,6.380952,6.096296,5.155455,5.190123,4.243704,5.888235,...,5.600000,4.515789,2.300000,5.140741,5.300000,5.580117,4.699220,6.200000,4.005882,6.000000
2022-11-27 23:59:40,4.305263,4.482353,5.288235,4.721053,6.380952,6.095712,5.155455,5.190476,4.241481,5.888235,...,5.600000,4.515789,2.300000,5.137037,5.300000,5.581871,4.699805,6.200000,4.005882,6.000000
2022-11-27 23:59:50,4.305263,4.482353,5.296078,4.721053,6.380952,6.094932,5.155455,5.190476,4.240000,5.888235,...,5.600000,4.515789,2.300000,5.133333,5.300000,5.584211,4.700000,6.200000,4.005882,6.000000


In [72]:
sensors_reindexed.dropna(inplace=True)
sensors_reindexed

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:01:00,6.711111,5.109630,5.192130,5.588325,6.758754,5.562434,5.437475,7.551852,6.749673,5.333951,...,7.077160,6.500000,3.245833,7.106173,7.718301,5.215595,7.022896,7.110136,6.780507,7.733333
2022-11-25 00:01:10,6.716898,5.124444,5.203704,5.585990,6.764646,5.571252,5.437222,7.555967,6.751852,5.318519,...,7.080247,6.500000,3.248148,7.112346,7.722658,5.235088,7.026263,7.114035,6.775634,7.735859
2022-11-25 00:01:20,6.721528,5.136296,5.212963,5.584122,6.769360,5.578307,5.437020,7.559259,6.753595,5.306173,...,7.082716,6.500000,3.250000,7.117284,7.726144,5.250682,7.028956,7.117154,6.771735,7.737879
2022-11-25 00:01:30,6.729630,5.157037,5.229167,5.580853,6.777609,5.590653,5.436667,7.565021,6.756645,5.290741,...,7.087037,6.500000,3.253241,7.125926,7.732244,5.277973,7.033670,7.122612,6.764912,7.741414
2022-11-25 00:01:40,6.733102,5.165926,5.236111,5.579452,6.781145,5.595944,5.436515,7.567490,6.757952,5.285185,...,7.088889,6.500000,3.254630,7.129630,7.734858,5.289669,7.035690,7.124951,6.761988,7.742929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:20,4.304678,4.484096,5.268627,4.721053,6.380952,6.097661,5.159226,5.188889,4.248889,5.887800,...,5.600000,4.515789,2.300000,5.149383,5.300000,5.576023,4.697856,6.203472,4.005882,6.001235
2022-11-27 23:59:30,4.305263,4.482571,5.282353,4.721053,6.380952,6.096296,5.155455,5.190123,4.243704,5.888235,...,5.600000,4.515789,2.300000,5.140741,5.300000,5.580117,4.699220,6.200000,4.005882,6.000000
2022-11-27 23:59:40,4.305263,4.482353,5.288235,4.721053,6.380952,6.095712,5.155455,5.190476,4.241481,5.888235,...,5.600000,4.515789,2.300000,5.137037,5.300000,5.581871,4.699805,6.200000,4.005882,6.000000
2022-11-27 23:59:50,4.305263,4.482353,5.296078,4.721053,6.380952,6.094932,5.155455,5.190476,4.240000,5.888235,...,5.600000,4.515789,2.300000,5.133333,5.300000,5.584211,4.700000,6.200000,4.005882,6.000000


In [73]:
with open('proj5_ex05.pkl', 'wb') as f5:
    pickle.dump(sensors_reindexed, f5)